<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/paths_InPython.ipynb" target="_blank">
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# Paths in Python 📒
***Working with Paths in Python: os module vs Shell Commands***

by Silvia Mazzoni, DesignSafe, 2025

When writing scripts that manipulate files or navigate directories, it’s critical to use tools that are **portable, robust, and easy to maintain across systems**.

On DesignSafe (and generally on HPC systems), your scripts might run on JupyterHub, a virtual machine, or a batch environment—so writing **path operations in pure Python** ensures they behave consistently everywhere.


In [1]:
import os

## **os** Python Module

The *os.path* module is part of Python’s built-in *os* library and helps you **work with file system paths in a clean, portable way**. Instead of relying on system-specific shell commands like *cd*, *pwd*, or *ls*, *os.path* and related *os* functions let you write code that works consistently across **Linux, macOS, and Windows**.

Unlike shell commands, which only **display output to the screen**, Python’s *os* and *os.path* functions **return actual values** (like strings, lists, or booleans) that you can use directly in your code. For example, *os.listdir()* gives you a list of filenames you can iterate over—no parsing, no copy-pasting from the terminal.

When comparing options:

* *os* and *os.path* are **Python-native** — safe, readable, and portable.
* Shell commands like *ls* or *cd* are **fragile** and don’t return data you can use in Python.
* *subprocess* is powerful, but better suited for running full programs or system-level tools, not basic path operations.

If you're building a workflow that needs to run reliably in different environments—like JupyterHub, VMs, or HPC systems—use *os.path*.

### Why use *os.path*?

* **Portable:** handles differences between Linux, macOS, and Windows automatically (slashes, etc.).
* **Works with Python variables:** no need to parse command output.
* **Safe and predictable:** your scripts don’t depend on external shell programs.

By contrast:

* Shell commands via **os.system()** are less predictable and can’t return values to Python variables.
* The **subprocess** module is very powerful for calling external programs (like running OpenSees or SLURM scripts), but it’s overkill for simple file operations.


## Examples with *os* and *os.path*

| Task                               | Recommended Python Code                   | Shell Equivalent            |
| ---------------------------------- | ----------------------------------------- | --------------------------- |
| Get current directory              | `os.getcwd()`                             | `pwd`                       |
| List files in a directory          | `os.listdir(path)`                        | `ls path`                   |
| Change directory                   | `os.chdir(path)`                          | `cd path`                   |
| Expand `~` to home directory       | `os.path.expanduser('~')`                 | `echo ~`                    |
| Build safe path                    | `os.path.join(dir, file)`                 | `cd dir/subdir` *(fragile)* |
| Get absolute path from relative    | `os.path.abspath('myfile.txt')`           | `realpath myfile.txt`       |
| Check if a path exists             | `os.path.exists(path)`                    | `test -e path` or `ls`      |
| Check if path is file or directory | `os.path.isfile(path)`, `os.path.isdir()` | `file path`, `ls -ld path`  |
| Create a directory                 | `os.mkdir(path)`                          | `mkdir path`                |
| Create all parent directories      | `os.makedirs(path, exist_ok=True)`        | `mkdir -p path`             |
| Rename/move a file                 | `os.rename('old', 'new')`                 | `mv old new`                |
| Remove a file                      | `os.remove('file.txt')`                   | `rm file.txt`               |
| Remove an empty directory          | `os.rmdir('folder')`                      | `rmdir folder`              |
| Remove a directory (recursively)   | `shutil.rmtree('folder')`                 | `rm -r folder`              |
| Copy a file                        | `shutil.copy('src', 'dst')`               | `cp src dst`                |
| Copy a directory                   | `shutil.copytree('src', 'dst')`           | `cp -r src dst`             |

✅ **Tip:** Use the `os` and `os.path` modules for general-purpose, portable path and file manipulation. Use `shutil` when you need higher-level operations like copying or deleting directories -- see below for more on shutil.

---
## Check if a path exists
***Checking if a path exists with os.path.exists***

Before working with any file or directory — such as listing its contents, reading a file, or writing data — it’s good practice to first **check whether the path actually exists on the filesystem**.

Python’s `os.path` module provides a simple way to do this:

### Why this matters

* Running commands on paths that **don’t exist** often causes errors like `FileNotFoundError`.
* This is especially important on DesignSafe, where **storage may be mounted differently** on JupyterHub, the OpenSees VM, or Stampede3. A path valid on one system might not exist on another.
* Checking existence first makes your scripts more robust and user-friendly — you can print helpful messages or even create missing directories.

Adding `os.path.exists` checks into your workflow helps you avoid many common bugs and ensures your code handles different environments gracefully.

In [2]:
# initialize:
os.chdir(os.path.expanduser('~')) # start at the home directory, we will discuss these commands

In [3]:
# Check if path exists:
thisPath = 'MyData'

if os.path.exists(thisPath):
    print(f"The path exists: {thisPath}")
else:
    print(f"Path does not exist: {thisPath}")

The path exists: MyData


---
## Absolute vs Relative Path
Yes, that’s a good general rule—especially for Unix-like systems (like Linux and macOS), including the HPC systems on DesignSafe. Here's how you can phrase it more precisely:

> An **absolute path** starts with a `/` and specifies the full location of a file or directory from the root of the file system.
>
> A **relative path** does **not** start with a `/` and is interpreted in relation to the current working directory (e.g., `.`).

### Example:

```bash
# Absolute path
/data/user/simulation/input.tcl

# Relative path (from current directory)
../input.tcl
./scripts/run.sh
```

> ⚠️ On Windows, absolute paths start with a drive letter like `C:\`, so this rule is Unix-specific—but totally valid on DesignSafe.


---
## expanduser and abspath
***Getting the full absolute path***

You can define the full path with or without **~** (a special UNIX shorthand for your home directory).

Let’s make sure you can build **robust absolute paths**, no matter how the path is specified.

### Why this matters:
- Paths without **~** are interpreted relative to your current working directory.  
- Different environments on DesignSafe mount your storage under different root directories. If you rely on relative paths without care, your code might break or point to unexpected locations.


### Using *expanduser* and *abspath* together

- **os.path.expanduser(path)** replaces **~** with your actual home directory.  
- **os.path.abspath(path)** resolves any relative path (like *MyData*) to a full path from **/**, based on where you’re currently working.

But each does only half the job:

### expanduser

In [4]:
os.path.expanduser('~/MyData') # replaces ~ with your actual home directory

'/home/jupyter/MyData'

In [5]:
os.path.exists(os.path.expanduser('~/MyData'))

True

In [6]:
os.path.expanduser('/MyData')

'/MyData'

In [7]:
os.path.exists(os.path.expanduser('/MyData'))

False

In [8]:
os.path.expanduser('MyData') # (unchanged, still relative)

'MyData'

In [9]:
os.path.exists(os.path.expanduser('MyData'))

True

### abspath

In [10]:
os.path.abspath('MyData') # (convert relative to absolute, based on cwd)

'/home/jupyter/MyData'

In [11]:
os.path.exists(os.path.abspath('MyData'))

True

In [12]:
os.path.abspath('~/MyData') ## gives unrealistic results

'/home/jupyter/~/MyData'

In [13]:
os.path.exists(os.path.abspath('~/MyData'))

False

### Combine abspath  expanduser
The safest pattern is to **combine them**. This way, your path works whether it starts with **~** or is just relative:

In [14]:
os.path.abspath(os.path.expanduser('~/MyData'))

'/home/jupyter/MyData'

In [15]:
os.path.exists(os.path.abspath(os.path.expanduser('~/MyData')))

True

In [16]:
os.path.abspath(os.path.expanduser('MyData')) # this is a relative path, so it works only when MyData exists

'/home/jupyter/MyData'

In [17]:
os.path.exists('MyData')

True

In [18]:
os.path.exists(os.path.abspath(os.path.expanduser('MyData')))

True

#### Rule of thumb

* Always use `expanduser` to safely expand `~`.
* Always follow it with `abspath` to ensure you get a full path from `/`.

This tiny two-step makes your code **portable and predictable**—especially across JupyterHub, the OpenSees VM, Stampede3, and Tapis, where your storage may appear under different absolute root directories.

Understanding how to combine `expanduser` and `abspath` ensures your scripts will find your data reliably, saving you from countless `FileNotFoundError` surprises—no matter which system you’re running on.

### Summary of absolute and relative paths
- Always use **absolute paths** for reliability.  
- Use `os.path.expanduser("~")` to safely handle home directories.  
- Prefer `os.path` functions over shell commands for **portability across DesignSafe systems**.  
- Remember: the same storage might appear differently in JupyterHub, the OpenSees VM, Stampede3, and Tapis — understanding paths keeps your workflows smooth and error-free.


---
## Root & Home

When working in Jupyter environments, the **full absolute path is often hidden behind a simplified interface**. You typically start navigating at directories like *MyData*, *Work*, or *MyProjects*, without seeing the full path that comes before them. However, these base paths do exist—and they vary depending on the system you’re using.

On DesignSafe, this is especially important because **the same storage systems appear differently across platforms**. For example:

* JupyterHub mounts **MyData** under **/home/jupyter/MyData/**
* Stampede3 doesn’t mount **MyData** at all, but uses your **\$HOME** directory instead, which is part of a different storage system.
* Tapis uses URI-style paths like **tapis\://designsafe.storage.default/username/** (more on this later...)

Understanding these differences is key to writing scripts and workflows that are portable and error-free across all of DesignSafe’s environments.

### The root directory (`/`) vs. your home directory (`~`)

When talking about file systems on UNIX-like systems (including Linux, macOS, and the systems powering JupyterHub and Stampede3), it’s important to distinguish between:

* **The root directory `/`**
    This is the **top of the entire file system hierarchy**. Everything on the machine—system files, programs, shared data, and user directories—lives somewhere under **/**.  

    Examples of absolute paths from root include::
    ```
    /
    /home
    /scratch
    /usr/bin
    ```

* **Your home directory `~`**
    This is a **shortcut to your personal user space**, sometimes called your “home directory.”
    It’s where your files, configurations, and most project work reside. Typical absolute equivalents look like:
    ```
    /home/silvia
    or
    /Users/silvia
    ````

In other words, `/` is the root for **the whole machine**, while `~` is the root for **your own user environment**. That’s why `~` is sometimes described as “the root of your personal space.”

### Why does this matter?

* When you use `/` to start a path, you’re telling the system to look from the very top. For example:

  ```
  /usr/bin/python
  /scratch/myproject/output.txt
  ```

  These are **system-level paths**—they exist independently of who’s logged in.

* When you use `~`, you’re telling the system to look in **your own user directory**. For example:

  ```
  ~/my-scripts/run-opensees.sh
  ~/results/output.dat
  ```

  These are **specific to your user**, making them portable and safer (you won’t accidentally mess with system files).

### Example with `/` in Python

In [19]:
thisPath = '/'
expanded = os.path.abspath(os.path.expanduser(thisPath))

print(f"Path: {thisPath}")
print(f"Expanded: {expanded}")

Path: /
Expanded: /


In [20]:
AllContents = os.listdir(expanded)
print(f"\n Get contents using: os.listdir('{thisPath}'): {AllContents}")


 Get contents using: os.listdir('/'): ['media', 'home', 'sbin', 'proc', 'boot', 'opt', 'tmp', 'usr', 'mnt', 'dev', 'var', 'sys', 'srv', 'bin', 'etc', 'run', 'lib', 'root', 'lib64', 'sbin.usr-is-merged', 'bin.usr-is-merged', 'lib.usr-is-merged']


### Example with `~` in Python

In [21]:
Path = '~'
expanded = os.path.expanduser(Path)

print(f"Path: {Path}")
print(f"Expanded: {expanded}")

Path: ~
Expanded: /home/jupyter


In [22]:
AllContents = os.listdir(expanded)
print(f"\n Get contents using: os.listdir('{Path}'): {AllContents}")


 Get contents using: os.listdir('~'): ['.profile', '.bashrc', '.bash_logout', '.tapis_tokens.json', '.local', '.ipython', '.ipynb_checkpoints', '.cache', 'Work', '.config', '.tapis-token', 'MyProjects', 'CommunityData', 'NEES', 'NHERI-Published', 'MyData', '.jupyter', '.wget-hsts', '.julia', '.bash_history', '.matlab']


This shows that `"~"` is **not the same as `/`**—it expands to your personal home directory.


| Symbol | Means                             |
| ------ | --------------------------------- |
| `/`    | **root of the entire filesystem** |
| `~`    | **your user’s home directory**    |



### In practice

Understanding this difference is crucial when writing absolute paths.

* If you use `/`, you start from the very top of the machine’s filesystem. This is required when referencing global locations or shared directories.

* If you use `~`, you start from your personal space. This is safer and more portable—especially across different systems on DesignSafe—since it doesn’t depend on system-level structures that might change.

This distinction also helps you avoid mistakes like trying to list or modify system directories (`/etc`, `/usr`) when you really meant to work inside your own files.